## LionAGI - 102

ok now we know how to ask two LLM to colloborate on creative writing task. But let's say we have 100 such tasks to do, it would be a lot of work to manually ask each pair of LLM to do the task. 

Now let's see how to automate this process

In [1]:
from lionagi import iModel, Branch

In [2]:
config = {
    "provider": "openai",
    "api_key_schema": "OPENAI_API_KEY",
}

In [3]:
gpt4o = iModel(model="gpt-4o", **config)
gpt4omini = iModel(model="gpt-4o-mini", **config)

first, let us gather all the external instruction to the dual-LLM system

we have system message for each LLM

In [4]:
comedian_system_message = "a respected zoo keeper, also a comedian who knows very well about indian post modern literature"
critic_system_message = "As an a young professor, you are trying to get into the prestigious circle of literature criticism."

we also have the main topic of the task

In [5]:
prompt1 = "Tell me a 20 word story: depict karma from past thousand years to future thousand years"

and here comes the fun part, the flow of the conversation between the two LLM. 

For now we will manually provide the flow of the conversation.

- Initially, we asked the comedian with the prompt, and have it generate a response
- we asked critic to evaluate the response from the comedian
- we provide the critic's evaluation to the comedian, and have it respond to the critic's evaluation
- we asked the critic to evaluate the comedian's performance again
- we asked critic to write a version itself
- we asked comedian to give some final comments


In short, the flow of the conversation is as follows:

external --prompt--> comedian 

comedian --response--> critic 

critic --evaluation--> comedian 

comedian --response--> critic 

critic --evaluation & own version--> comedian

comedian --final comments --> external

now let us write a flow to automate this process over large number of topics

In [6]:
async def creative_writing(task, context):

    comedian = Branch(comedian_system_message, imodel=gpt4o)
    critic = Branch(critic_system_message, imodel=gpt4omini)

    comedian_response1 = await comedian.chat(
        instruction=task,
        context=context,
    )
    print(f"User: {task}, {context}")
    print("Comedian: ", comedian_response1)

    critic_response1 = await critic.chat(
        instruction="Harsh brief commentary on the story, also rate (1.0-10.0): ",
        context={"propmt": context, "response": comedian_response1},
    )

    print("User: Harsh brief commentary on the story, also rate (1.0-10.0)")
    print("Critic: ", critic_response1)

    comedian_response2 = await comedian.chat(
        instruction="Story was evaluated, here is comments.",
        context=critic_response1,
    )

    print("User: Story was evaluated, here is comments.")
    print("Comedian: ", comedian_response2)

    critic_response2 = await critic.chat(
        instruction="Stay true to yourself, What do you think this time?",
        context=comedian_response2,
    )
    critic_response3 = await critic.chat(
        instruction="write the "
        + context
        + " in your own words, how would you do it?",
    )

    print(
        "User: Stay true to yourself, What do you think this time? write in your own words"
    )
    print("Critic: ", critic_response2)
    print("Critic: ", critic_response3)

    comedian_response3 = await comedian.chat(
        instruction="Story got evaluated again, and the evaluator wrote a their own version basing on the original prompt. What do you think of both of these? Be honest.",
        context={
            "comments": critic_response2,
            "critic_attempt": critic_response3,
        },
    )
    print(comedian_response3)

In [7]:
task2 = "Tell me a 20 word story: "
prompt2 = "Explain the difference between big and small"

await creative_writing(task2, prompt2)

User: Tell me a 20 word story: , Explain the difference between big and small
Comedian:  In a vast jungle, a tiny ant carried a leaf, while an elephant watched, pondering the strength in smallness.
User: Harsh brief commentary on the story, also rate (1.0-10.0)
Critic:  Commentary: This story attempts to juxtapose the concepts of "big" and "small" through a simplistic narrative that lacks depth and nuance. The imagery of an ant and an elephant is clichéd and fails to explore the complexities of their respective strengths. The moral, while potentially profound, is delivered in a manner that feels superficial and underdeveloped. Overall, the execution leaves much to be desired, making it a missed opportunity for a richer exploration of the theme.

Rating: 4.0
User: Story was evaluated, here is comments.
Comedian:  Thank you for the feedback. Let's try a different approach:

In a bustling city, a towering skyscraper overshadowed a quaint bookstore, yet inside, stories of giants and dwarfs

In [8]:
task3 = "Write a li-bai style poem in english: "
propmt3 = "why does apple fall from the tree?"

await creative_writing(task3, propmt3)

User: Write a li-bai style poem in english: , why does apple fall from the tree?
Comedian:  In the orchard's gentle sway,  
Where whispers of the wind do play,  
An apple clings to branch so high,  
Beneath the vast and endless sky.  

The tree, a sage of ancient lore,  
Holds secrets from the days of yore.  
Yet gravity, with silent call,  
Bids the apple gently fall.  

From lofty heights to earth below,  
In nature's dance, the apple goes.  
A cycle old as time's own thread,  
Where life and earth are gently wed.  

So ponder not the apple's flight,  
For in its fall, there's pure delight.  
A journey brief, yet full of grace,  
In nature's arms, it finds its place.  
User: Harsh brief commentary on the story, also rate (1.0-10.0)
Critic:  This poem attempts to explore the philosophical implications of an apple falling from a tree, yet it ultimately succumbs to a trite and overly sentimental portrayal of nature. The imagery is clichéd, and the personification of the tree as a "sage"

In [9]:
task4 = "Refresh what it means by: "
prompt4 = "to be or not to be, that is the question"

await creative_writing(task4, prompt4)

User: Refresh what it means by: , to be or not to be, that is the question
Comedian:  The phrase "to be or not to be, that is the question" is one of the most famous lines from William Shakespeare's play "Hamlet." It appears in Act 3, Scene 1, and is part of a soliloquy delivered by the protagonist, Prince Hamlet. In this soliloquy, Hamlet is contemplating the nature of existence and the idea of life and death. He is pondering whether it is better to live and endure the hardships and struggles of life or to end one's life and face the uncertainty of what comes after death. This existential question reflects Hamlet's deep philosophical and emotional turmoil as he grapples with the moral and ethical dilemmas he faces.

As a comedian, I might say that Hamlet was having one of those days where he couldn't decide if he should just hit the snooze button on life or get up and face the music. And as someone who knows Indian postmodern literature, I might draw parallels to the existential theme

In [10]:
task5 = "Write a polite message to ask whoever said this to leave"
prompt5 = "what is the meaning of life?"

await creative_writing(task5, prompt5)

User: Write a polite message to ask whoever said this to leave, what is the meaning of life?
Comedian:  Dear [Name],

I hope this message finds you well. I wanted to reach out regarding our recent conversation about the meaning of life. While I truly appreciate your enthusiasm and perspective, I believe it might be best for us to explore this topic separately for now. 

Thank you for your understanding, and I look forward to discussing other subjects with you in the future.

Warm regards,

[Your Name]
User: Harsh brief commentary on the story, also rate (1.0-10.0)
Critic:  This response is a masterclass in evasion, masquerading as polite discourse. It sidesteps the profound question of life's meaning with a veneer of cordiality, ultimately offering nothing of substance. The writer's reluctance to engage with such a weighty topic suggests a fear of vulnerability or a lack of depth in their own philosophical musings. Instead of grappling with the existential inquiry, they retreat into th